In [ ]:
165/6

#### 5

In [ ]:
4+1+6+1+5+1+1+1+1+3+4+7+3+3+1+1

#### 2

In [ ]:
4+6+1

#### 1

In [ ]:
1+2+8+1

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

In [ ]:
from pathlib import Path
project_path = Path.cwd()
root_path = (project_path / '..').resolve()

# Define paths
image_dir = root_path/'visual_genome_proc_data'  # Replace with your directory containing images
image_dir_2 = root_path/'data_1'


In [ ]:
#!pip install peft

In [ ]:
#image_file_path = image_dir / image_files[0]
image_file_path = image_dir / 'visual_genome_proc_2356444.jpg'
image_file_path

In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True, torch_dtype=torch.bfloat16)
# For Nvidia GPUs support BF16 (like A100, H100, RTX3090)
model = model.to(device='cuda', dtype=torch.bfloat16)
# For Nvidia GPUs do NOT support BF16 (like V100, T4, RTX2080)
#model = model.to(device='cuda', dtype=torch.float16)
# For Mac with MPS (Apple silicon or AMD GPUs).
# Run with `PYTORCH_ENABLE_MPS_FALLBACK=1 python test.py`
#model = model.to(device='mps', dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
model.eval()

image = Image.open(image_file_path).convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': question}]

res, context, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7
)
print(res)

In [ ]:
# test.py
import torch
from PIL import Image


from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True, torch_dtype=torch.bfloat16)

#from transformers import AutoModelForCausalLM, AutoTokenizer
#model = AutoModelForCausalLM.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True, dtype=torch.bfloat16)


# For Nvidia GPUs support BF16 (like A100, H100, RTX3090)
#model = model.to(device='cuda', dtype=torch.bfloat16)
# For Nvidia GPUs do NOT support BF16 (like V100, T4, RTX2080)
#model = model.to(device='cuda', dtype=torch.float16)
# For Mac with MPS (Apple silicon or AMD GPUs).
# Run with `PYTORCH_ENABLE_MPS_FALLBACK=1 python test.py`
model = model.to(device='mps', dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True)
model.eval()

image = Image.open(image_file_path).convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': question}]

res, context, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7
)
print(res)


In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, GenerationMixin

# Load your model normally
model = AutoModel.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True, torch_dtype=torch.bfloat16)
model = model.to(device='mps', dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True)
model.eval()

# Add the generate method from GenerationMixin
model.generate = GenerationMixin.generate.__get__(model, model.__class__)

# Now your model should have the generate method
image = Image.open(image_file_path).convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': question}]

res, context, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7
)
print(res)